# Exercise 1 b)

In [72]:
# optimization algorithm to minimize unconstrained function
from scipy.optimize import newton
from math import sqrt
import pandas as pd

import numpy as np
import matplotlib as plt

import plotly.express as px
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

Function to minimize:<br>

$f(x) = x_1^2 + x_2^2 + 16x_3^2$ <br>
$\nabla f(x) = (2x_1, 8x_2, 32x_3)^T$

constraint: <br>

$h(x) = x_1x_2 - 1$



In [73]:
# function to minimize
f = lambda x: x[0]**2 + 4*x[1]**2 + 16*x[2]**2
# its gradient/jacobian
JacF = lambda x: [2*x[0], 8*x[1], 32*x[2]]
# norm of its gradient/jacobian
normJacF = lambda x: np.linalg.norm(JacF(x))
# constraint
h = lambda x: x[0] * x[1] - 1

Values to pay attention to 

In [74]:
x_star1 = (sqrt(2),1/sqrt(2),0) # minimum 1
x_star2 = (-sqrt(2),-1/sqrt(2),0) # minimum 1
print(f"{x_star1=}")
print(f"{x_star2=}")
print(f"{f(x_star1)=}")
print(f"{f(x_star2)=}")
print("Lagrangian multiplier: 4") # lagrange mult. associated with the problem

x_star1=(1.4142135623730951, 0.7071067811865475, 0)
x_star2=(-1.4142135623730951, -0.7071067811865475, 0)
f(x_star1)=4.0
f(x_star2)=4.0
Lagrangian multiplier: 4


# Penalty Method

$p(x) = \frac{1}{2}h(x)^Th(x) = ||h(x)||_2^2 = \frac{1}{2}(x_1x_2 - 1)^T (x_1x_2 - 1)$

$P(x,\mu) = f(x) + \mu p(x) =  x_1^2 + 4x_2^2 + 16x_3^2 + \frac{\mu}{2}(x_1x_2 - 1)^T (x_1x_2 - 1)$

In [75]:
# feasibility penalization function
p = lambda x: 0.5 * h(x)**2
# Merit function
P = lambda mu: lambda x: f(x) + mu*p(x)
JacP = lambda mu: lambda x: np.array([2*x[0] + mu*(x[0]*x[1]**2 -x[1]), \
                                      8*x[1] + mu*(x[1]*x[0]**2 -x[0]), 32*x[2]])

## minimize

In [76]:
data = pd.DataFrame(columns=['iteration','x', 'f(x)',"||f'(x)||",'P(x)','p(x)','h(x)','mu', 'mu*h(x)','log10||x_k-x_min||']).set_index('iteration')

# initial conditions
x_start = np.array([20,30,15])
mu_start = 5 # if we change to 2 for example, the method fails to converge

i=0 # iteration
x = x_start
mu = mu_start
mu_update_coef = 2

# start optimization loop
max_iter = 200
while i <= max_iter:
    data.loc[i] = [x, f(x), normJacF(x), P(mu)(x), p(x), h(x), mu, mu*h(x),np.log10(np.linalg.norm(x-x_star1))]
    i += 1 
    x = newton(JacP(mu), x, maxiter=5000)
    mu *= mu_update_coef
data.tail(3)

,x,f(x),||f'(x)||,P(x),p(x),h(x),mu,mu*h(x),log10||x_k-x_min||
iteration,,,,,,,,,
198,"[1.4140626753249295, 0.7071822327608044, 0.0]",4,6.32496,4.0,0.0,0,2008672555323737844427452615426453253152753742...,0,-3.772884
199,"[1.4140626753249295, 0.7071822327608044, 0.0]",4,6.32496,4.0,0.0,0,4017345110647475688854905230852906506305507484...,0,-3.772884
200,"[1.4140626753249295, 0.7071822327608044, 0.0]",4,6.32496,4.0,0.0,0,8034690221294951377709810461705813012611014968...,0,-3.772884


## Plotting 

How is the convergence of f(x) to the minimum?

In [77]:
def save_plotly_fig_x_pure(fig, x_start,mu_start,mu_update):
    fig.write_image(f"media/Ex1_pure_xk_plot__initial_condition_x={x_start}_mu={mu_start}_mu_update_rule=muX{mu_update}.png")
    
def save_plotly_fig_muh_pure(fig, x_start,mu_start,mu_update):
    fig.write_image(f"media/Ex1_pure_muh_plot__initial_condition_x={x_start}_mu={mu_start}_mu_update_rule=muX{mu_update}.png")
    
def save_plotly_fig_x_augmented(fig, x_start,mu_start,mu_update):
    fig.write_image(f"media/Ex1_aug_xk_plot__initial_condition_x={x_start}_mu={mu_start}_mu_update_rule=muX{mu_update}.png")
    
def save_plotly_fig_muh_augmented(fig, x_start,mu_start,mu_update):
    fig.write_image(f"media/Ex1_aug_muh_plot__initial_condition_x={x_start}_mu={mu_start}_mu_update_rule=muX{mu_update}.png")

In [78]:
layout = dict(title_text='Convergence of x to the minimum in Pure penalization method', title_x=0.5, xaxis_title='iterations', yaxis_title='log10 ||x_k - x_min1||')

fig = px.scatter(data['log10||x_k-x_min||'])
fig.update_layout(layout,showlegend=False) # add titles, remove unecessary legend

save_plotly_fig_x_pure(fig,x_start,mu_start,mu_update_coef)
fig.show()

Conclusions:

After 20 iterations the method achieves 5 decimal points of precision of the minimum estimate!

Now let's look at the term $\mu \cdot h(x)$ and check if it converges to the lagrange multiplier associated with this problem (which is equal to -4).

In [79]:
# title information
layout = dict(title_text='behaviour of mu * h(x)', title_x=0.5, xaxis_title='iterations', yaxis_title='value')
# horizontal line with true value for lambda
lagrange_multiplier_line = dict(type= 'line', y0= -4, y1= -4, 
                                x0= 0, x1= max_iter,
                                line=dict(color="Red",width=4))

fig = px.scatter(data['mu*h(x)'][1:])
fig.update_layout(layout, showlegend=False) # add titles, remove unecessary legend
fig.update_layout(shapes=[lagrange_multiplier_line]) # add horizontal line with true value

save_plotly_fig_muh_pure(fig, x_start, mu_start, mu_update_coef)
fig.show()

Conclusions:

From the 1st iteration, the method achieves 8 decimals of precision and stays very close to that for the following iterations. It is unnable to converge to -4.

# Augmented Lagrangian Method

$
\begin{equation}
\begin{aligned}
L(x,\lambda,\mu) &= f(x) + \lambda^Th(x) + \frac{\mu}{2}h(x)^Th(x) \\
&= x_1^2 + x_2^2 + 16x_3^2 + \lambda^T(x_1x_2 - 1) + \frac{\mu}{2} (x_1x_2 - 1)^T (x_1x_2 - 1)
\end{aligned}
\end{equation}
$

In [65]:
# feasibility penalization function
p = lambda x: 1/2 * h(x)**2
# Merit function
L = lambda mu,lamb: lambda x: f(x) + lamb*h(x) + mu*p(x)
JacL = lambda mu,lamb: lambda x: np.array([2*x[0] + (lamb*x[1]) + mu*(x[0]*x[1]**2 -x[1]), 
                                           8*x[1] + (lamb*x[0]) + mu*(x[1]*x[0]**2 -x[0]), 32*x[2]])
normJacL = lambda mu,lamb: lambda x: np.linalg.norm(JacL(mu,lamb)(x))

## minimize

In [68]:
data = pd.DataFrame(columns=['iteration','x', 'f(x)',"||f'(x)||",'L(x)',"||L'(x)||",'p(x)','h(x)','mu', 'lambda','log10||x_k-x_min||']).set_index('iteration')

# initial conditions
j=0 # iteration
x = np.array([20,30,15])

mu = 5 
mu_update_coef = 2

lamb = 2

last_saved_f = 0 # iteration to compare to in order to update mu when function has decreased enough

# start optimization loop
max_iter = 20
while j <= max_iter: 
    data.loc[j] = [x, f(x), normJacF(x), L(mu,lamb)(x),normJacL(mu,lamb)(x), p(x), h(x), mu, lamb, np.log10(np.linalg.norm(x-x_star1))]
    x = newton(JacL(mu,lamb), x, maxiter=10000, disp=False)
    
    # update mu
    if f(x) < 0.8 * data.loc[last_saved_f,'f(x)']:
        mu *= mu_update_coef
        last_saved_f = j
    lamb = lamb + mu * h(x) # update lambda
    j += 1
data.tail(5)

,x,f(x),||f'(x)||,L(x),||L'(x)||,p(x),h(x),mu,lambda,log10||x_k-x_min||
iteration,,,,,,,,,,
16,"[1.4142135623941945, 0.7071067811755952, 0.0]",4,6.324555,4.0,1.837180e-10,1.620640e-25,-5.69322e-13,20,-4,-10.623924
17,"[1.4142135623867467, 0.7071067811794612, 0.0]",4,6.324555,4.0,1.188656e-10,6.784897e-26,-3.68372e-13,20,-4,-10.813012
18,"[1.4142135623819279, 0.7071067811819626, 0.0]",4,6.324555,4.0,7.690911e-11,2.840891e-26,-2.38365e-13,20,-4,-11.002099
19,"[1.4142135623788101, 0.707106781183581, 0.0]",4,6.324555,4.0,4.976238e-11,1.189036e-26,-1.5421e-13,20,-4,-11.191179
20,"[1.4142135623767929, 0.7071067811846282, 0.0]",4,6.324555,4.0,3.218572e-11,4.958786e-27,-9.9587e-14,20,-4,-11.380268


## Plotting

In [70]:
layout = dict(title_text='Convergence of x to the minimum in Augmented Lagrangian method', title_x=0.5, xaxis_title='iterations', yaxis_title='log10 ||x_k - x_min||')
fig = px.scatter(data['log10||x_k-x_min||'])
fig.update_layout(layout, showlegend=False) # add titles, remove unecessary legend

save_plotly_fig_x_augmented(fig,x_start,mu_start,mu_update_coef)
fig.show()

Conclusions:

This method converges a lot better than the pure one. There's also a large step that increases the accuracy by 6 decimal points in iteration 4.

How does the lambda estimate behave over the iterations?

In [71]:
# title information
layout = dict(title_text='behaviour of mu * h(x) in Augmented Lagrangian', title_x=0.5, xaxis_title='iterations', yaxis_title='mu*h(x)')
# horizontal line with true value for lambda
lagrange_multiplier_line = dict(type= 'line', y0= -4, y1= -4, 
                                x0= 0, x1= max_iter,
                                line=dict(color="Red",width=4))

fig = px.scatter(data['lambda'][1:])
fig.update_layout(layout, showlegend=False) # add titles, remove unecessary legend
fig.update_layout(shapes=[lagrange_multiplier_line]) # add horizontal line with true value

save_plotly_fig_muh_augmented(fig, x_start, mu_start, mu_update_coef)
fig.show()